In [1]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np

# Next cell contains all parameters, which need to be set

In [2]:
#Configure this block
embedding_path = r"D:\\tagged&tags\\"
embedding_name = 'treeTaggedwTags_150w'
outlocation = "D:\\TWTS1\\"
New_emb_Name_path = outlocation+embedding_name + '_combined'#later added with String _strategyX
unfiltered_vocab = Vocabulary()
unfiltered_vocab.load(r"C:\Users\weso\Desktop\gauntlet\vocabs\unfilteredvocab_treeTaggedwTags")
dimensions = 500

In [3]:
import numpy as np
import tensorflow as tf
id_dict = {}
word_dict = {}

matrix = []
full_path = os.path.join(embedding_path,embedding_name)
with open(full_path, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

309746


In [4]:
#find all variants of a word, without the tag rep
def group_all_words(id_dict):
    groups = {}
    for word in id_dict:
        #print(word)
        if is_tagged_word(word):
            #print('is tagged')
            baseword = word.split(chr(4))[0]
            #print(baseword)
            if baseword in groups:
                groups[baseword].append(word)
            else:
                groups[baseword] = [word]
        elif word.startswith(chr(4)):#exclude tags
            pass
        else:
            if word in groups:
                groups[word].append(word)
            else:
                groups[word] = [word]
    return groups

def is_tagged_word(word):
    return not word.startswith(chr(4)) and str(chr(4)) in word

groups = group_all_words(id_dict)
print(len(groups))
print(groups['duck'])
print(groups['the'])
print(groups['bank'])
print('---------')
print(str(groups)[:100])
print(groups["splashdown"])

243605
['duck\x04nnp', 'duck\x04vb', 'duck\x04nn']
['the\x04dt', 'the\x04nnp', 'the\x04rb', 'the\x04nfp', 'the\x04']
['bank\x04nnp', 'bank\x04jj', 'bank\x04vb', 'bank\x04nn']
---------
{'chromatic': ['chromatic\x04nn', 'chromatic\x04nnp', 'chromatic\x04jj'], 'domini': ['domini\x04nnp'
['splashdown\x04nnp', 'splashdown\x04nn']


# helper

In [5]:
def get_count(word):
    return unfiltered_vocab.word_frequency[word]

In [6]:
def save_new_embedding(embedding,path):
    with open(path,'w+',encoding='utf8') as file:
        for id,word in enumerate(embedding):
            if(id % 1000 == 0):
                print(id,len(embedding))
            
            file.write(word)
            vector = embedding[word]
            for coord in vector:
                file.write(' '+str(coord))
            file.write('\n')

# Strategy 1 weighted Sum

In [7]:
#requires tag rep
def mult_strat_weighted(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:

        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if chr(4) in word:
                if(word.startswith(chr(4))):#no rep for tags
                    continue
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count) * (matrix_copy[id_word])
            else:#only in mixed embedding add the base unscaled
                emb_sum += (get_count(word)/base_count) * matrix_copy[id_dict[word]]
        emb[baseword] = emb_sum
    return emb
    

In [8]:
emb1 = mult_strat_weighted(groups,False)
save_new_embedding(emb1,New_emb_Name_path+"_Strat1")

0 243604
1000 243604
2000 243604
3000 243604
4000 243604
5000 243604
6000 243604
7000 243604
8000 243604
9000 243604
10000 243604
11000 243604
12000 243604
13000 243604
14000 243604
15000 243604
16000 243604
17000 243604
18000 243604
19000 243604
20000 243604
21000 243604
22000 243604
23000 243604
24000 243604
25000 243604
26000 243604
27000 243604
28000 243604
29000 243604
30000 243604
31000 243604
32000 243604
33000 243604
34000 243604
35000 243604
36000 243604
37000 243604
38000 243604
39000 243604
40000 243604
41000 243604
42000 243604
43000 243604
44000 243604
45000 243604
46000 243604
47000 243604
48000 243604
49000 243604
50000 243604
51000 243604
52000 243604
53000 243604
54000 243604
55000 243604
56000 243604
57000 243604
58000 243604
59000 243604
60000 243604
61000 243604
62000 243604
63000 243604
64000 243604
65000 243604
66000 243604
67000 243604
68000 243604
69000 243604
70000 243604
71000 243604
72000 243604
73000 243604
74000 243604
75000 243604
76000 243604
77000 243604

# Strategy 2 AddTagRep

In [9]:
#requires tag rep
def add_tag_rep_strat(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:
        
        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            if chr(4) in word:
                if word.endswith(chr(4)):
                    baseword = word.split(chr(4))[0]
                else:
                    baseword ,tag  = word.split(chr(4))
                
                id_word = id_dict[word]
                try:
                    id_tag  = id_dict[chr(4)+tag]
                    emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] + matrix_copy[id_tag])
                except:
                    emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word])
            else:#only in mixed embedding add the base unscaled
                emb_sum += matrix_copy[id_dict[word]] * (get_count(word)/base_count)
        emb[baseword] = emb_sum
    return emb

In [10]:
emb3 = add_tag_rep_strat(groups,False)
save_new_embedding(emb3,New_emb_Name_path+"_Strat2")

0 243604
1000 243604
2000 243604
3000 243604
4000 243604
5000 243604
6000 243604
7000 243604
8000 243604
9000 243604
10000 243604
11000 243604
12000 243604
13000 243604
14000 243604
15000 243604
16000 243604
17000 243604
18000 243604
19000 243604
20000 243604
21000 243604
22000 243604
23000 243604
24000 243604
25000 243604
26000 243604
27000 243604
28000 243604
29000 243604
30000 243604
31000 243604
32000 243604
33000 243604
34000 243604
35000 243604
36000 243604
37000 243604
38000 243604
39000 243604
40000 243604
41000 243604
42000 243604
43000 243604
44000 243604
45000 243604
46000 243604
47000 243604
48000 243604
49000 243604
50000 243604
51000 243604
52000 243604
53000 243604
54000 243604
55000 243604
56000 243604
57000 243604
58000 243604
59000 243604
60000 243604
61000 243604
62000 243604
63000 243604
64000 243604
65000 243604
66000 243604
67000 243604
68000 243604
69000 243604
70000 243604
71000 243604
72000 243604
73000 243604
74000 243604
75000 243604
76000 243604
77000 243604